In [19]:
import json

In [1]:
import boto3

In [2]:
session = boto3.session.Session()

In [28]:
sts = session.client('sts')
sts.get_caller_identity()

{'UserId': 'AROA5TWTPH6BVIRFIP7Q6:eoin.shanaghy@fourtheorem.com',
 'Account': '935672627075',
 'Arn': 'arn:aws:sts::935672627075:assumed-role/AWSReservedSSO_DevAdministratorAccess_427a06cd5ebe9968/eoin.shanaghy@fourtheorem.com',
 'ResponseMetadata': {'RequestId': 'a177ead6-88e3-4a3f-bfc2-c2f3433b788b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a177ead6-88e3-4a3f-bfc2-c2f3433b788b',
   'content-type': 'text/xml',
   'content-length': '519',
   'date': 'Thu, 23 Nov 2023 21:03:03 GMT'},
  'RetryAttempts': 0}}

In [29]:
lambda_client = session.client('lambda')

In [30]:
paginator = lambda_client.get_paginator('list_functions')

In [31]:
iter = paginator.paginate()
function_names = [f['FunctionName'] for f in page['Functions'] for page in iter]
measurement_function_names = [fn for fn in function_names if fn.startswith('perf_') and fn.endswith('_import_measure')]
measurement_function_names

['perf_zip_Python39_import_measure',
 'perf_image_Python39_import_measure',
 'perf_zip_layers_Python38_import_measure',
 'perf_zip_Python310_import_measure',
 'perf_zip_Python311_import_measure',
 'perf_zip_layers_Python311_import_measure',
 'perf_image_Python311_import_measure',
 'perf_zip_layers_Python310_import_measure']

In [32]:
responses = {fn: lambda_client.invoke(FunctionName=fn, InvocationType="RequestResponse") for fn in measurement_function_names}    

In [33]:
[json.load(val['Payload']) for val in responses.values()]

[{'_frozen_importlib_external': '2484',
  'zipimport': '791',
  'encodings': '2583',
  'encodings.utf_8': '266',
  '_signal': '404',
  'encodings.latin_1': '554',
  'io': '1295',
  'site': '15472',
  'base64': '18244',
  'datetime': '1947',
  'json': '2316',
  'numpy': '939036',
  'pandas': '3281855',
  'pyarrow.parquet': '219630',
  'aws_lambda_powertools': '122974',
  'aws_xray_sdk.core': '80902',
  'aws_xray_sdk.ext.pynamodb.patch': '1031',
  'aws_xray_sdk.ext.requests.patch': '1485',
  'requests': '108',
  'aws_xray_sdk.ext.botocore.patch': '171378',
  'aws_xray_sdk.ext.pymysql.patch': '715',
  'aws_xray_sdk.ext.pymongo.patch': '772',
  'aws_xray_sdk.ext.pg8000.patch': '545',
  'aws_xray_sdk.ext.psycopg2.patch': '1635',
  'psycopg2': '105',
  'aws_xray_sdk.ext.sqlalchemy_core.patch': '9539',
  'aws_xray_sdk.ext.httpx.patch': '1019',
  'aws_xray_sdk.ext.sqlite3.patch': '3069',
  'aws_xray_sdk.ext.mysql.patch': '673',
  'aws_xray_sdk.ext.aiobotocore.patch': '16585'},
 {'_frozen_impor